In [ ]:
# -*- coding: utf-8 -*-
""" BEP ANALYSIS
Created on Fri Jan 12 21:24:29 2024

@author: X1 Carbon Gen 7
"""

import matplotlib.pyplot as plt

Units = 1000
SellPrice = 2000
Variable = 800
Fixed = float(input('Fixed cost for this year is: ' ))
TargetProfit = float(input('Target Profit for this year is: ' ))

# -------------------------------------------- in Power Point
# Your units is: 1000
# Your selling price per unit is: 2000
# Variable cost is: 800
# Fixed cost for this year is: 1200000
# Target Profit for this year is: 100000

# -------------------------------------------- in Excel
# Your units is: 2000
# Your selling price per unit is: 60
# Variable cost is: 45
# Fixed cost for this year is: 240000
# Target Profit for this year is: 100000

#PYTHON IS EASY LANGUAGES PROGRAMMING FOR HUMAN 

#C+ C++ JAVA PYSCAL => HARD NOT HUMAN => EASY FRIENDLY COMPUTER

#COMPUTER---------------------------------HUMAN
#C+--------------------------------------PYTHON


CM_Ratio = SellPrice-Variable  # Contribution Margin Ratio 
CM = Units*SellPrice - Units*Variable # Contribution Margin

print (f'\n\n Your Contribution Margin per unit is:  {CM:.2f}' )
NC = CM - Fixed # Net income
print (f'\n\n Your Net Income per unit is:  {NC:.2f}' )
NumberofUnits = (Fixed + TargetProfit) / CM_Ratio

BEP = Fixed/(SellPrice-Variable) # Break-Even Point

BEPS = Fixed/(CM_Ratio/SellPrice) # Break-Even Point in Sales

print (f'\n Your break even point in units is:  {BEP:.2f} ' 
       + f'\n\n Your break even point in total sales dollars is:  {BEPS:.2f}')


# Fixed Costs
plt.plot([0, Fixed],[Fixed, Fixed + 2 * BEP], color='red', label='Fixed Costs')
# Total Costs (Fixed + Variable)
plt.plot([0, 2 * BEP],[Fixed, Fixed + Variable * 2 * BEP], color='orange',label='Total Costs (Fixed + Variable)')
# Revenue
plt.plot([0, 2 * BEP],[0, SellPrice * 2 * BEP], color='green',label='Revenue')

# Break-Even Point
plt.plot([BEP]*2,[0, BEPS], color='black', linestyle='--', lw=1 ,marker = 'o', markerfacecolor = 'red', markersize = 6)
plt.plot([0, BEP],[BEPS]*2, color='black', linestyle='--', lw=1 ,marker = 'o', markerfacecolor = 'red', markersize = 6)

plt.scatter(NumberofUnits, Fixed+BEPS+TargetProfit, c ="blue", linewidths = 2, marker ="*", label='Target Profit: 100.000 (USD)')

plt.legend()
plt.title('Break-Even Analysis by Rinez')
plt.ylabel('Sales in Dollars')
plt.xlabel('Units Sold')

plt.ylim(bottom=0)
plt.xlim(left=0)
plt.xlim(right=2*BEP)
plt.show()


In [1]:
from datetime import datetime, timedelta, date
import pandas as pd
import pyodbc 

# =============================================================================
# Connect to SQL Server
# =============================================================================
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)


# =============================================================================
# Transform Data with SQL Server
# =============================================================================
df_RFM = pd.read_sql_query('''WITH RFM_Base 
    AS
    (
        SELECT a.Customer_ID, b.Customer_Name AS CustomerName,
        DATEDIFF(DAY, MAX(a.Order_Date), CONVERT(DATE, GETDATE())) AS Recency_Value,
        COUNT(DISTINCT a.Order_Date) AS Frequency_Value,
        ROUND(SUM(a.Sales), 2) AS Monetary_Value
      FROM [vti_dataset].[dbo].[sales] AS a
      INNER JOIN [vti_dataset].[dbo].[customer] AS b ON a.Customer_ID = b.Customer_ID
      GROUP BY a.Customer_ID, b.Customer_Name
    )
    -- SELECT * FROM RFM_Base
    , RFM_Score 
    AS
    (
      SELECT *,
        NTILE(5) OVER (ORDER BY Recency_Value DESC) as R_Score,
        NTILE(5) OVER (ORDER BY Frequency_Value ASC) as F_Score,
        NTILE(5) OVER (ORDER BY Monetary_Value ASC) as M_Score
      FROM RFM_Base
    )
    -- SELECT * FROM RFM_Score
    , RFM_Final
    AS
    (
    SELECT *,
      CONCAT(R_Score, F_Score, M_Score) as RFM_Overall
      -- , (R_Score + F_Score + M_Score) as RFM_Overall1
      -- , CAST(R_Score AS char(1))+CAST(F_Score AS char(1))+CAST(M_Score AS char(1)) as RFM_Overall2
    FROM RFM_Score
    )
    -- SELECT * FROM RFM_Final
    SELECT f.*, s.Segment
    FROM RFM_Final f
    JOIN [segment scores] s ON f.RFM_Overall = s.Scores
    ; ''', conn)

# =============================================================================
# Transform Data with Python
# =============================================================================

df_sales = pd.read_sql_query('SELECT * FROM [vti_dataset].[dbo].[sales] ', conn)
df_customers = pd.read_sql_query('SELECT * FROM [vti_dataset].[dbo].[customers] ', conn)
df_segmentScores = pd.read_sql_query('SELECT * FROM [vti_dataset].[dbo].[segment scores]', conn)

df_sales.info()
df_customers.info()
df_segmentScores.info()

df_RFM.info()
df_RFM['CustomerName'].nunique()
# =============================Recency=========================================
 
df_sales['Order_Date'] = pd.to_datetime(df_sales['Order_Date'])

df_user = df_customers[['Customer_ID','Customer_Name']]
df_user.columns = ['CustomerID','CustomerName']

df_maxPurchase = df_sales.groupby('Customer_ID').Order_Date.max().reset_index()
df_maxPurchase.columns = ['CustomerID','MaxPurchaseDate']

df_maxPurchase['Recency Value'] = (datetime.now() - df_maxPurchase['MaxPurchaseDate']).dt.days

df_RFMOverall = pd.merge(df_user, df_maxPurchase[['CustomerID','Recency Value']], on='CustomerID')

df_RFMOverall['Recency Value'].describe()

# =============================Frequency=======================================

df_frequency = df_sales.groupby('Customer_ID').Order_Date.nunique().reset_index()
df_frequency.columns = ['CustomerID','Frequency Value']

df_RFMOverall = pd.merge(df_RFMOverall, df_frequency, on='CustomerID')

df_RFMOverall['Frequency Value'].describe()

# =============================Monetery Value==================================

df_monetery = df_sales.groupby('Customer_ID').Sales.sum().reset_index()

df_monetery.columns = ['CustomerID','Monetery Value']

df_RFMOverall = pd.merge(df_RFMOverall, df_monetery, on='CustomerID')

df_RFMOverall['Monetery Value'].describe()

# =============================RFM-Score==================================

df_RFMOverall['R-Score'] = pd.qcut(df_RFMOverall['Recency Value'], 5, labels=[5,4,3,2,1]).astype(str)
df_RFMOverall['F-Score'] = pd.qcut(df_RFMOverall['Frequency Value'], 5, labels=[1,2,3,4,5]).astype(str)
df_RFMOverall['M-Score'] = pd.qcut(df_RFMOverall['Monetery Value'], 5, labels=[1,2,3,4,5]).astype(str)

df_RFMOverall['RFM_Overall'] = df_RFMOverall['R-Score'] + df_RFMOverall['F-Score'] + df_RFMOverall['M-Score']

df_RFMOverall.info()
df_segmentScores.info()

# =============================Segment name==================================

df_segmentScores['Scores'] = df_segmentScores['Scores'].astype(str)

df_RFMOverall = df_RFMOverall.merge(df_segmentScores, how='inner', left_on = 'RFM_Overall', right_on = 'Scores')

df_RFMOverall = df_RFMOverall.iloc[:, :-1]
# df_RFMOverall.drop('Scores', axis=1, inplace=True)










C:\Users\Admin-s\AppData\Local\Temp\ipykernel_2680\2471510822.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_RFM = pd.read_sql_query('''WITH RFM_Base
C:\Users\Admin-s\AppData\Local\Temp\ipykernel_2680\2471510822.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales = pd.read_sql_query('SELECT * FROM [vti_dataset].[dbo].[sales] ', conn)
C:\Users\Admin-s\AppData\Local\Temp\ipykernel_2680\2471510822.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customers = pd.read_sql_query('SELECT * FROM [vti_datase

DatabaseError: Execution failed on sql 'SELECT * FROM [vti_dataset].[dbo].[customers] ': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'vti_dataset.dbo.customers'. (208) (SQLExecDirectW)")